In [1]:
# Dependencies
import tweepy
import twitter_service as ts
from collections import defaultdict
from config import consumer_key, consumer_secret, access_token, access_token_secret

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [2]:
# Twitter API Keys
consumer_key = consumer_key
consumer_secret = consumer_secret
access_token = access_token
access_token_secret = access_token_secret


In [9]:
def main():
    # Counter
    #counter = 1

    # Variables for holding sentiments
    sentiments = defaultdict(list)

    # Variable for max_id
    oldest_tweet = None
    
    api = ts.setup_connection(consumer_key, consumer_secret, access_token, access_token_secret)
    public_tweets = ts.grab_tweets(api, '@CNN', 100)
    for index, tweet in enumerate(public_tweets):
        # Run Vader Analysis on each tweet
        results = analyze_tweets(tweet)
        sentiments["Handle"].append(tweet["user"]["screen_name"])
        sentiments["Date"].append(tweet["created_at"])
        sentiments["Compound"].append(results["compound"])
        sentiments["Positive"].append(results["pos"])
        sentiments["Neutral"].append(results["neu"])
        sentiments["Negative"].append(results["neg"])
        sentiments["Tweets Ago"].append(index + 1)
        sentiments["Tweet Text"].append(tweet["text"])
    print(sentiments)

In [11]:
def analyze_tweets(tweet):
    # Run Vader Analysis on each tweet
    return analyzer.polarity_scores(tweet["text"])
        

In [12]:
main()

setting up connection...
grabbing your tweets...
defaultdict(<class 'list'>, {'Handle': ['CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN', 'CNN'], 'Date': ['Mon Apr 23 03:00:22 +0000 2018', 'Mon Apr 23 02:30:05 +0000 2018', 'Mon Apr 23 02:00:14 +0000 2018', 'Mon Apr 23 01:45:07 +0000 2018', 'Mon Apr 23 01:30:09 +0000 2018', 'Mon Apr 23 01:15:32 +0000 2018